In [1]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
!pip install torchdiffeq
from torchdiffeq import odeint_adjoint, odeint
import numpy as np
from scipy import io
import argparse
import os
import pickle
import matplotlib.pyplot as plt
import zipfile
from timeit import default_timer
from google.colab import drive
import h5py

In [2]:
drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
directory = "drive/MyDrive/Fourier Neural Operator/data/"#"spring-runs_train/" #/mnt/data/datasets/physical_system_benchmark/experiment_output/spring/spring/run/data_gen/spring-runs_train-spring-n1000-t805-n0_00001/"
zip = zipfile.ZipFile(directory + "NavierStokes_V1e-5_N1200_T20.zip")
zip.extractall()

Mounted at /content/drive


In [3]:
npz = io.loadmat("NavierStokes_V1e-5_N1200_T20.mat")
# npz = h5py.File("NavierStokes_V1e-5_N1200_T20.mat")
u = torch.tensor(npz["u"], device=device, dtype=torch.float32)
a = torch.tensor(npz["a"], device=device, dtype=torch.float32)
print(u.shape, a.shape)
nb_trajs = 1200
x_width = 64
y_width = 64
# initial = torch.tensor(a, dtype = torch.float32)
# final = torch.tensor(x, dtype = torch.float32)
# p_exact = torch.tensor(p1, dtype = torch.float32)
# t = torch.linspace(0,1,nb_times, dtype = torch.float32)

torch.Size([1200, 64, 64, 20]) torch.Size([1200, 64, 64])


In [4]:
# directory = "drive/MyDrive/Fourier Neural Operator/data"#"spring-runs_train/" #/mnt/data/datasets/physical_system_benchmark/experiment_output/spring/spring/run/data_gen/spring-runs_train-spring-n1000-t805-n0_00001/"
# npz = np.load(directory + "burgers_data_R10.mat")
# nb_trajs = 50
# start_traj = 0
# nb_times = 10205
# x_width = 125
# p1, q1, dqdt1, dpdt1, t1 = np.zeros((nb_trajs,x_width,nb_times)), np.zeros((nb_trajs,x_width,nb_times)), np.zeros((nb_trajs,x_width,nb_times)), np.zeros((nb_trajs,x_width,nb_times)), np.zeros((nb_trajs,x_width,nb_times))
# for i in range(start_traj, start_traj + nb_trajs):
#   p1[i,:,:], q1[i,:,:], dqdt1[i,:,:], dpdt1[i,:,:], t1[i,:,:] = npz[npz.files[5*i+0]].T, npz[npz.files[5*i+1]].T, npz[npz.files[5*i+2]].T, npz[npz.files[5*i+3]].T, npz[npz.files[5*i+4]].T
# q_exact = torch.tensor(q1, dtype = torch.float32)
# p_exact = torch.tensor(p1, dtype = torch.float32)
# t = torch.tensor(t1[0], dtype = torch.float32)
# exact = torch.cat((q_exact.reshape(nb_trajs,x_width,nb_times,1), p_exact.reshape(nb_trajs,x_width,nb_times,1)), dim=-1)

In [5]:
test_trajs = 200
data = u #torch.cat((a.reshape(nb_trajs,x_width,1), u.reshape(nb_trajs,x_width,1)), dim=-1)
train_loader = DataLoader(data[:(nb_trajs-test_trajs)], batch_size=16)
test_loader  = DataLoader(data[(nb_trajs-test_trajs):], batch_size=16)

In [6]:
print(u.shape)
print(nb_trajs, test_trajs)
print(u[(nb_trajs-test_trajs):].shape)

torch.Size([1200, 64, 64, 20])
1200 200
torch.Size([200, 64, 64, 20])


In [7]:
class Lyapunov(nn.Module):
  def __init__(self, in_nodes, layers, activation):
    super(Lyapunov, self).__init__()
    self.in_nodes = in_nodes
    self.layers = layers
    self.act = activation
    self.w0 = nn.Linear(in_nodes, in_nodes)
    self.w1 = nn.Linear(in_nodes, 1)
    self.u1 = nn.Parameter(torch.Tensor(1, in_nodes))
    nn.init.xavier_uniform_(self.u1)
    self.g0 = self.g(torch.zeros(in_nodes))
  def g(self, x):
    z = self.act(self.w0(x))
    z = self.act(self.w1(x)  + F.linear(z, self.u1.exp()))
    return z
  def forward(self, x):
    return self.act(self.g(x) - self.g0) + torch.sum(x**2, dim=1).reshape(-1,1)/100
# class Lyapunov(nn.Module):
#   def __init__(self, in_nodes, layers, activation):
#     super(Lyapunov, self).__init__()
#     self.in_nodes = in_nodes
#     self.layers = layers
#     self.act = activation
#     self.W = [nn.Linear(in_nodes, in_nodes)]
#     self.U = []
#     for i in range(layers):
#       self.W.append(nn.Linear(in_nodes, in_nodes))
#       self.U.append(nn.Linear(in_nodes, in_nodes))
#     self.W.append(nn.Linear(in_nodes, 1))
#     self.U.append(nn.Linear(in_nodes, 1))
#     self.g0 = self.g(torch.zeros(in_nodes))
#   def g(self, x):
#     z = self.act(self.W[0](x))
#     for i in range(self.layers):
#       z = self.act(self.W[i+1](x) + self.U[i](z)**2)
#     z = self.act(self.W[self.layers+1](x) + self.U[self.layers](z)**2)
#     return z
#   def forward(self, x):
#     return self.act(self.g(x) - self.g0) + torch.norm(x, p=2)/100

In [8]:
class ODEFunc(nn.Module):
    def __init__(self, in_nodes, hid_nodes, v_layers, v_act):
        super(ODEFunc, self).__init__()
        self.fc0 = nn.Linear(in_nodes+1, hid_nodes)
        self.fc1 = nn.Linear(hid_nodes, in_nodes)
        self.V = Lyapunov(in_nodes, v_layers, v_act)
    def forward(self, t, x):
        x = x.requires_grad_(True)
        tt = torch.ones_like(x[...,:1], requires_grad=True)*t
        xt = torch.cat((tt, x), dim=-1)
        fx = self.fc0(xt)
        fx = F.gelu(fx)
        fx = self.fc1(fx)
        # Vx = self.V(x)
        # grad_Vx = torch.autograd.grad(Vx, x, torch.ones(Vx.shape), create_graph=True, allow_unused=True, retain_graph=True)[0]
        # alpha = 1
        # fx = fx - grad_Vx*torch.relu(torch.sum(grad_Vx*fx,dim=1).reshape(-1,1) + alpha*Vx)/torch.sum(grad_Vx**2, dim=1).reshape(-1,1)
        return fx

In [9]:
class NODE(nn.Module):
    def __init__(self, in_nodes, hid_nodes, tol, v_layers, v_act):
        super(NODE, self).__init__()
        self.ode = ODEFunc(in_nodes, hid_nodes, v_layers, v_act)
        self.fc0 = nn.Linear(2, in_nodes)
        self.fc1 = nn.Linear(in_nodes, 2)
        self.tol = tol
    def forward(self, x):
        x = x.requires_grad_(True)
        x = x.reshape(x.shape[0], -1)
        x = self.fc0(x)
        x0 = x
        x = odeint(self.ode, x, torch.tensor([0., 1.], device=x.device), atol=self.tol, rtol=self.tol)[-1]
        x = self.fc1(x)
        return x

In [10]:
skip_size = 1
nb_times = u.shape[-1]//skip_size
skip_size_2 = 2
Nt = nb_times//skip_size
Nx = (x_width//skip_size_2)*(y_width//skip_size_2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# model = NODE(32, 128, 1e-5, 3, nn.Softplus()).to(device)
latent_channels = 64
hidden_layers = 128
odefunc = ODEFunc(latent_channels, hidden_layers, 2, nn.ReLU())
prelayer = nn.Linear(Nx, latent_channels)
postlayer = nn.Linear(latent_channels, Nx)
params = list(odefunc.parameters()) + list(prelayer.parameters()) + list(postlayer.parameters())
optim = torch.optim.Adam(params, lr=1e-3)
loss_fn = nn.MSELoss()
tol = 1e-5
mse_train_losses = []
mse_test_losses = []
lyap_train_losses = []
lyap_test_losses = []
accuracies = []

cpu


In [11]:
epochs = 500

In [ ]:
alpha = 5
grads = []
grads_added = False
filter = []
for i in range(epochs):
    t = default_timer()
    lyap_train_loss = 0
    mse_train_loss = 0
    mse_test_loss = 0
    lyap_test_loss = 0
    for x in train_loader:
        x = x[:,::skip_size_2,::skip_size_2]
        x0 = x[:,:,:,0].to(device).requires_grad_(True)
        x0 = x0.reshape(x.shape[0], -1)
        z0 = prelayer(x0)
        zt = odeint_adjoint(odefunc, z0, torch.linspace(0,1,Nt), atol=tol, rtol=tol)
        pred = postlayer(zt)
        pred = pred.permute(2, 0, 1)
        pred = pred.reshape(x.shape)
        # print(pred.shape, x.shape)
        # loss = loss_fn(pred,x[:,::skip_size,:])
        loss = torch.tensor(0.0, requires_grad=True, device=device)
        # print(pred.shape, x.shape)
        loss = loss_fn(pred, x)
        # loss_now = loss_fn(pred[0], x0[0])
        # for j in range(1,pred.shape[0]):
        #   loss_prev = loss_now
        #   loss_now = loss_fn(pred[j], x0[j])
        #   loss = loss + torch.relu(loss_now + (alpha-1)*loss_prev)
        optim.zero_grad()
        loss.backward()
        for k,p in enumerate(params):
          if not grads_added:
            grads.append(torch.abs(p.grad))
            filter.append(torch.ones(p.grad.shape))
          else:
            grads[k] = grads[k] + torch.abs(p.grad)
        grads_added = True
        for j, p in enumerate(params):
          p.grad = p.grad*filter[j]
        optim.step()
        lyap_train_loss += loss.item()
        mse_train_loss += loss_fn(pred, x).item()
    for x in test_loader:
        x = x[:,::skip_size_2,::skip_size_2]
        x0 = x[:,:,:,0].to(device).requires_grad_(True)
        x0 = x0.reshape(x.shape[0], -1)
        z0 = prelayer(x0)
        zt = odeint(odefunc, z0, torch.linspace(0,1,Nt), atol=tol, rtol=tol)
        pred = postlayer(zt)
        pred = pred.reshape(x.shape)
        # pred = pred.permute(2, 0, 1, 3)
        loss = torch.tensor(0.0, requires_grad=True, device=device)
        loss_now = loss_fn(pred[0], x[0])
        for j in range(1,pred.shape[0]):
          loss_prev = loss_now
          loss_now = loss_fn(pred[j], x[j])
          loss = loss + torch.relu(loss_now + (alpha-1)*loss_prev)
        lyap_test_loss += loss.item()
        mse_test_loss += loss_fn(pred, x).item()
    mse_train_loss /= len(train_loader)
    mse_test_loss /= len(test_loader)
    lyap_train_loss /= len(train_loader)
    lyap_test_loss /= len(test_loader)
    mse_train_losses.append(mse_train_loss)
    mse_test_losses.append(mse_test_loss)
    lyap_train_losses.append(lyap_train_loss)
    lyap_test_losses.append(lyap_test_loss)
    if i%10==0 or i ==epochs-1:
      print(i, default_timer()-t, lyap_train_loss, lyap_test_loss, mse_train_loss, mse_test_loss)
      # for p,g,f in zip(params,grads,filter):
      #   j = torch.argmax(f/g, keepdim=True)
      #   # print(j, g.flatten()[j])
      #   f2 = f.flatten()
      #   f2[j] = 0
      #   f = f2.reshape(f.shape)
        # print(p[j//p.shape[1],j%p.shape[1]])
        # p[j//p.shape[1],j%p.shape[1]].requires_grad = False
        # p.requires_grad=False
        # print(j)
        # print(p.shape, g.shape)
        # print(p.flatten()[j], g.flatten()[j])
# for g in grads:
#   print(g)


0 58.064825641 0.9651555882559882 132.08886542687048 0.9651555882559882 1.8686963594876802
10 48.665431890000036 0.6317740640943013 134.78252880389874 0.6317740640943013 1.8822081272418683
20 54.615793077000035 0.6279995479280986 135.7416446392353 0.6279995479280986 1.8951440774477446
30 53.630765870999994 0.6263979626080346 136.41779327392578 0.6263979626080346 1.9044406505731435
40 49.83357465600011 3.640031996227446 374.9520791860727 3.640031996227446 5.3114705085754395
50 49.87935647600034 3.607192101932707 371.31331810584436 3.607192101932707 5.259929436903733
60 52.393155373000354 3.5669033262464733 366.8315194936899 3.5669033262464733 5.196400000498845
70 49.336546602 3.5191505522955033 361.5105813833383 3.5191505522955033 5.120907178291907
80 49.810406209000575 3.4642104705174765 355.41156710111176 3.4642104705174765 5.034278942988469
90 48.79366077400027 3.4025795497591536 348.65211017315204 3.4025795497591536 4.938144537118765
100 49.12293819500064 3.334982323268103 341.40216

In [ ]:
torch.save([lyap_train_losses, lyap_test_losses, mse_train_losses, mse_test_losses], "node_nse_losses.data")

In [ ]:
torch.save([prelayer.state_dict(), odefunc.state_dict(), postlayer.state_dict()], "node_nse_model.data")

In [ ]:
plt.plot(range(len(mse_train_losses)), np.array(mse_train_losses), label="MSE train loss")
plt.plot(range(len(mse_test_losses)), np.array(mse_test_losses), label="MSE test loss")
plt.yscale('log')
plt.legend()
plt.show()
plt.plot(range(len(lyap_train_losses)), np.array(lyap_train_losses), label="Lyap. train loss")
plt.plot(range(len(lyap_test_losses)), np.array(lyap_test_losses), label="Lyap. test loss")
plt.yscale('log')
plt.legend()
plt.show()
plt.plot(range(len(mse_test_losses)), np.abs(np.array(mse_test_losses)-np.array(mse_train_losses)), label="MSE difference")
plt.yscale('log')
plt.legend()
plt.show()
plt.plot(range(len(lyap_test_losses)), np.abs(np.array(lyap_test_losses)-np.array(lyap_train_losses)), label="Lyap. difference")
plt.legend()
plt.yscale('log')
plt.show()